Organize folder in the following way - in folder there are:
1. this script
2. file with spots coordinates, pre-filtered such that it containes only spots in good cells (not dividing, not on edge, not without nucleus)
3. folder called Cellpose_DONE/ that contains all tif nuclei mask images from Cellpose 

In [2]:
from skimage import io
import numpy as np
import pandas as pd
from glob import glob

In [2]:
# import tif file names in "files" object (3rd point in the instructions)
files = glob('Cellpose_DONE/*.tif') 
files

# ID = file_.split('/')[1][0:15] # from file name extract unique ID (donor, time, coverslip)

['Cellpose_DONE/D1_2HCD3CD28_05_25_50_cp_masks.tif',
 'Cellpose_DONE/D1_2HCD3CD28_06_24_48_cp_masks.tif',
 'Cellpose_DONE/D1_2HCD3CD28_01_23_48_cp_masks.tif']

In [3]:
# import file with spots coordinates of filtered cells into df object (2nd point in the instructions)
# this is an example for donor 1 - timepoint 2h, but a list for all donors/timepoints/cells can also be plugged in for a bulk analysis
df = pd.read_csv('Filtered_list_of_spots_coordinates_D1_2H.csv')
df.drop('Unnamed: 0', inplace=True, axis=1) # remove index column 
df

,ID,cytokine,Z_det,Y_det,X_det
0,D1_2HCD3CD28_01_Cell_CP_10,IFN,23,146,1991
1,D1_2HCD3CD28_01_Cell_CP_10,IFN,9,129,2000
2,D1_2HCD3CD28_01_Cell_CP_10,IFN,10,147,2049
3,D1_2HCD3CD28_01_Cell_CP_10,IFN,8,126,2027
4,D1_2HCD3CD28_01_Cell_CP_10,IFN,8,135,1998
...,...,...,...,...,...
5452,D1_2HCD3CD28_06_Cell_CP_96,TNF,49,1636,564
5453,D1_2HCD3CD28_06_Cell_CP_96,TNF,51,1610,552
5454,D1_2HCD3CD28_06_Cell_CP_96,TNF,30,1624,569
5455,D1_2HCD3CD28_06_Cell_CP_96,TNF,30,1629,524


Using negative indices (for z below kept stack) actually extracts a wrong value because it just goes from the other side,
so nuclei_mask[1, 189, 2000] is the same as nuclei_mask[-30, 189, 2000]
That's why we have to prevent tif intensity search for spots coordinates that fall below the cut stack.

In [4]:
# go to each spot, i.e., row by row through spots table; check ID, then open the corresponding nuclei mask tif;
# check if the coordinates of spot have non-zero intensity in the mask image (= if spot is in a nucleus);
# then record this information in a new column of spots table

spots_below_stack = 0
spots_above_stack = 0

# loop through the table of spots locations
for index, row in df.iterrows(): 

    # counter to track progress
    print(index)
    if (index-1)%1000 == 0:
        print(index-1, "out of", len(df))

    # find ID for the given row; note that this line may have to be adjusted depending on file naming
    row_ID = row['ID'][0:15] 
    print(row_ID)

    # find tif mask for the given ID (donor_timepoint_coverslip) among files and turn into string
    file_ = list(filter(lambda x: row_ID in x, files))
    print(file_)
    file_ = "".join(file_)   

    # load the image file; dimensions are (z-stack, 2034, 2034)
    nuclei_mask = io.imread(file_)
    print(nuclei_mask.shape)

    # from filename of tif cellpose mask extract from which stack the tif began (note: z-stack was clipped for purposes of Cellpose)
    begin_stack = int(file_.split('_')[-4]) 
    max_stack = int(file_.split('_')[-3]) - int(file_.split('_')[-4]) 
    print(begin_stack)
    print(max_stack)

    # extract coordinates x, y, z of spot (adjust for z stack clipping)
    x,y,z = row['X_det'], row['Y_det'], row['Z_det'] - begin_stack 
    x,y,z = int(x), int(y), int(z)

    # if z is below 0 (below the first stack) or above max_stack, then we're outside nucleus > assign 0
    # else assign value from tif image at given coordinates 
    if z < 0:
        df.loc[index,'nuclei_mask'] = 0
        spots_below_stack = spots_below_stack + 1
    elif z > max_stack:
        df.loc[index,'nuclei_mask'] = 0
        spots_above_stack = spots_above_stack + 1
    else:
        df.loc[index,'nuclei_mask'] = nuclei_mask[z,y,x]
        
    print("-------------------")

print("DONE!")

# save the table
df.to_csv("Spots_with_localization.csv")

0 out of 5457
1000 out of 5457
2000 out of 5457
3000 out of 5457
4000 out of 5457
5000 out of 5457
DONE!


In [5]:
# print overview statistics
print(len(df[df['nuclei_mask']>0]), "out of", len(df), "are in nucleus -", round(100*len(df[df['nuclei_mask']>0])/len(df), 1),"%")
print("We had", spots_below_stack , "spots below the cut stack -", round(100*spots_below_stack/len(df),1),"%")
print("We had", spots_above_stack , "spots above the cut stack -", round(100*spots_above_stack/len(df),1),"%")

2615 out of 5457 are in nucleus - 47.9 %
We had 1721 spots below the cut stack - 31.5 %
We had 56 spots above the cut stack - 1.0 %
